## Quick Results

This notebook is an extension to <code>main_analysis</code> in order to provide a quick way to reproduce results.

In [1]:
# Core
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats
import os
import subprocess
from __future__ import division
import pickle

def install_package(name):
    sudoPassword = ''
    command = 'pip install ' + name
    p = os.system('echo %s|sudo -S %s' % (sudoPassword, command))

# Stats
from statsmodels.regression import linear_model
import statsmodels.api as sm

# ML
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle


# OR
from skcriteria import Data, MIN, MAX
from skcriteria.madm import closeness, simple

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from ggplot import *



/Library/Python/2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [16]:
# MODELS
"""
Models = {'lda':get_LDA_scores, 
          'rf':get_RF_scores,
          'svc':get_SVC_scores,
          'log_reg':get_LogReg_scores,
          'nb':get_NB_scores,
          'knn':get_KNN_scores}
"""

# FEATURE SPACES
mixed_selection_features = pickle.load(open('../pickled/mixed_selection_features.pickle', 'rw'))
mixed_selection_1x_features = pickle.load(open('../pickled/mixed_selection_1x_features.pickle', 'rw'))
lasso_features = pickle.load(open('../pickled/lasso_features.pickle', 'rw'))
topsis_features = pickle.load(open('../pickled/topsis_features.pickle', 'rw'))
rf_features = pickle.load(open('../pickled/rf_features.pickle', 'rw'))
feature_space      = {  '../pickled/topsis_features':topsis_features, 
                         '../pickled/rf_features':rf_features, 
                         '../pickled/lasso_features':lasso_features,
                         '../pickled/mixed_selection_1x_features':mixed_selection_1x_features,
                         '../pickled/mixed_selection_features':mixed_selection_features     }

# FULL SPACE DATA SCORES
lda_full_scores     = pickle.load(open('../pickled/lda_full_scores.pickle', 'rw'))
rf_full_scores      = pickle.load(open('../pickled/rf_full_scores.pickle', 'rw'))
svm_full_scores     = pickle.load(open('../pickled/svm_full_scores.pickle', 'rw'))
log_reg_full_scores = pickle.load(open('../pickled/log_reg_full_scores.pickle', 'rw'))
nb_full_scores      = pickle.load(open('../pickled/nb_full_scores.pickle', 'rw'))
knn_full_scores     = pickle.load(open('../pickled/knn_full_scores.pickle', 'rw'))
full_scores         = {'lda':lda_full_scores,
                       'rf':rf_full_scores,
                       'svm':svm_full_scores,
                       'log_reg':log_reg_full_scores,
                       'nb':nb_full_scores,
                       'knn':knn_full_scores}

# DATA IN EACH SUBSPACE
data_per_space=pickle.load(open("../pickled/data_per_space.pickle", 'rw'))


# FULL SPACE C.I.
full_confidence_intervals = pickle.load(open("../pickled/full_confidence_intervals.pickle", "rw"))

# SUB SPACE DATA SCORES
lda_space_scores=pickle.load(open("../pickled/lda_space_scores.pickle", 'rw'))
rf_space_scores=pickle.load(open("../pickled/rf_space_scores.pickle", "rw"))
svm_space_scores=pickle.load(open("../pickled/svm_space_scores.pickle", "rw"))
log_reg_space_scores=pickle.load(open("../pickled/log_reg_space_scores.pickle", "rw"))
nb_space_scores=pickle.load(open("../pickled/nb_space_scores.pickle", "rw"))
knn_space_scores=pickle.load(open("../pickled/knn_space_scores.pickle", "rw"))
sub_scores = {'lda':lda_space_scores,
             'rf':rf_space_scores,
             'svm':svm_space_scores,
             'log_reg':log_reg_space_scores,
             'nb':nb_space_scores,
             'knn':knn_space_scores}

# SUB SPACE C.I.
sub_model_confidence_interval=pickle.load(open("../pickled/sub_model_confidence_interval.pickle", "rw") )

# INITIAL DATA
bigframe = pickle.load(open('../pickled/bigframe.pickle', 'rw'))
bigframe

In [18]:
#full_scores["lda"] vs. sub_scores["lda"]["mixed_selection_features"]
#full_scores["svm"] vs. sub_scores["svm"]["rf_features"]
#full_scores["log_reg"] vs. sub_scores["log_reg"]["topsis_features"]
#full_scores["nb"] vs. sub_scores["nb"]["rf_features"]
#full_scores["knn"] vs. sub_Scores["knn"]["rf_features"]
import scipy.stats as stats

print("(LDA) Is Reduced set more predictive than Full set?")
for k,v in sub_scores["lda"].items():    
    lda_pval=stats.ttest_ind(v,  # >=
                    full_scores["lda"], 
                    axis=0, equal_var=True)
    print(k,lda_pval[1]/2)

print("(SVM) Small P means 2nd > 1st")
for k,v in sub_scores["svm"].items(): 
    svm_pval=stats.ttest_ind(v,  # >=
                    full_scores["svm"], 
                    axis=0, equal_var=True)
    print(k, svm_pval[1]/2)

print("(Log_Reg) Small P means 2nd > 1st")
for k,v in sub_scores["log_reg"].items(): 
    log_reg_pval=stats.ttest_ind(v,  # >=
                    full_scores["log_reg"], 
                    axis=0, equal_var=True)
    print(k, log_reg_pval[1]/2)

print("(NB) Small P means 2nd > 1st")
for k,v in sub_scores["nb"].items(): 
    nb_pval=stats.ttest_ind(v,  # >=
                    full_scores["nb"], 
                    axis=0, equal_var=True)

    print(k, nb_pval[1]/2)
print("(KNN) Small P means 2nd > 1st")
for k,v in sub_scores["knn"].items(): 
    knn_pval=stats.ttest_ind(v,  # >=
                    full_scores["knn"], 
                    axis=0, equal_var=True)
    print(k,knn_pval[1]/2)
print("(RF) Small P means 2nd > 1st")
for k,v in sub_scores["rf"].items(): 
    rf_pval=stats.ttest_ind(v,  # >=
                    full_scores["rf"], 
                    axis=0, equal_var=True)
    print(k,rf_pval[1]/2)
# LDA topsis_features 0.380512957039085
# SVM lasso_features 0.21698345384201057
# Log_Reg mixed_selection_features 0.37566324944873475
# NB lasso_features 0.000000002 reject null fullspace is greater.
# KNN mixed_selection_features 0.34511637590581723

(LDA) Is Reduced set more predictive than Full set?
('topsis_features', 0.380512957039085)
('lasso_features', 0.098324070265796759)
('mixed_selection_1x_features', 0.028071975619860336)
('mixed_selection_features', 0.008654197441309408)
('rf_features', 0.05777661125381079)
(SVM) Small P means 2nd > 1st
('topsis_features', 0.00030688569037786356)
('lasso_features', 0.21698345384201057)
('mixed_selection_1x_features', 0.0086206004775333696)
('mixed_selection_features', 0.0048227973496748257)
('rf_features', 0.00018036814977609173)
(Log_Reg) Small P means 2nd > 1st
('topsis_features', 0.018826118300759656)
('lasso_features', 0.0046659740282534758)
('mixed_selection_1x_features', 0.17932548819272764)
('mixed_selection_features', 0.37566324944873475)
('rf_features', 0.19716495543953705)
(NB) Small P means 2nd > 1st
('topsis_features', 8.9513312699864532e-14)
('lasso_features', 2.3795197121061658e-09)
('mixed_selection_1x_features', 3.2311275394699811e-05)
('mixed_selection_features', 7.6513

In [ ]:
# Recap
# Feature Space

full_confidence_intervals={}
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m-h, m+h

for k,v in full_scores.items():
    
    full_confidence_intervals[k] = mean_confidence_interval(v)

full_confidence_intervals

In [9]:
feature_space.keys()

['pickled/mixed_selection_1x_features',
 'pickled/mixed_selection_features',
 'pickled/topsis_features',
 'pickled/lasso_features',
 'pickled/rf_features']

In [15]:
for k,v in feature_space.items():
    print(len(v))

46
33
91
28
38


{'knn': (0.89895714076062472, 0.99944752132321191),
 'lda': (0.45475543878895508, 0.71181409929122663),
 'log_reg': (0.83175376297037928, 0.99999040344149925),
 'nb': (0.49150592186912728, 0.57480891640797893),
 'rf': (0.85079240358872865, 0.93115190396781911),
 'svm': (0.9991957980838384, 0.9993300340753497)}